In [ ]:
!pip install -qU "numpy<2.0" "opencv-python-headless==4.10.*"
!pip install -qU ultralytics --no-deps
!pip install sahi
!pip install python-chess

In [ ]:
# CELL 1: Additional Functions and Classes
import cv2
import numpy as np
import json
import pandas as pd
from pathlib import Path
from collections import Counter, deque
from typing import List, Dict, Tuple, Optional, Deque
from dataclasses import dataclass
import chess
from ultralytics import YOLO

# Try to import SAHI (optional)
try:
    from sahi import AutoDetectionModel
    from sahi.predict import get_sliced_prediction
    SAHI_AVAILABLE = True
except ImportError:
    SAHI_AVAILABLE = False

# Configuration
CORNER_MODEL_PATH = "/kaggle/input/digiimg-model/pytorch/default/1/corner_detector/corner_detector/weights/corner_detect_best.pt"  # Update path as needed
PIECE_MODEL_PATH = "/kaggle/input/digiimg-model/pytorch/default/1/chess_models/chess_models/chess_detector_balanced/weights/best.pt"  # Update path as needed
BOARD_SIZE = 800
GRID_MARGIN = 35
WINDOW_SIZE = 30
CONFIDENCE_THRESHOLD = 0.7

# ============================================================================
# ChessBoardSystem Class
# ============================================================================
"""
Complete Chess Board Detection System
Combines: Corner Detection + Piece Detection (SAHI) + Square Grid Mapping (Phase 3)
"""

import cv2
import numpy as np
from pathlib import Path
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional
from ultralytics import YOLO
import json

try:
    from sahi import AutoDetectionModel
    from sahi.predict import get_sliced_prediction
    SAHI_AVAILABLE = True
except ImportError:
    print("[WARNING] SAHI not available. Install with: pip install sahi")
    SAHI_AVAILABLE = False


@dataclass
class PieceDetection:
    """Single piece detection"""
    class_id: int
    class_name: str
    confidence: float
    x_center: float  # Normalized
    y_center: float
    width: float
    height: float
    
    @property
    def color(self) -> str:
        if 'white' in self.class_name.lower():
            return 'white'
        elif 'black' in self.class_name.lower():
            return 'black'
        return 'unknown'
    
    @property
    def piece_type(self) -> str:
        return self.class_name.replace('white-', '').replace('black-', '')


class ChessBoardSystem:
    """
    Complete chess board detection and analysis system
    
    Features:
    - Corner detection for board localization
    - SAHI-based piece detection for small objects
    - Square grid mapping (Phase 3)
    - Board state extraction
    """
    
    def __init__(
        self,
        corner_model_path: str,
        piece_model_path: str,
        board_size: int = 800,
        use_sahi: bool = True,
        slice_size: int = 640,
        overlap_ratio: float = 0.2,
        grid_margin: int = 35  # Margin to account for board borders
    ):
        """
        Initialize chess board system
        
        Args:
            corner_model_path: Path to corner detection model
            piece_model_path: Path to piece detection model
            board_size: Size of extracted board (800x800)
            use_sahi: Use SAHI for piece detection
            slice_size: Size of each slice for SAHI
            overlap_ratio: Overlap between slices
            grid_margin: Margin in pixels to account for board borders (default: 20)
        """
        self.board_size = board_size
        self.grid_margin = grid_margin
        self.use_sahi = use_sahi and SAHI_AVAILABLE
        self.slice_size = slice_size
        self.overlap_ratio = overlap_ratio
        
        self.locked_rotation = None  # Allow locking orientation to prevent flipping
        
        # Load corner model
        print(f"Loading corner model: {corner_model_path}")
        self.corner_model = YOLO(corner_model_path)
        
        # Load piece model
        print(f"Loading piece model: {piece_model_path}")
        if self.use_sahi:
            print(f"  Using SAHI sliced inference")
            print(f"  Slice size: {slice_size}, Overlap: {overlap_ratio}")
            self.piece_model = AutoDetectionModel.from_pretrained(
                model_type='yolov8',
                model_path=piece_model_path,
                confidence_threshold=0.3,
                device='cpu'
            )
        else:
            self.piece_model = YOLO(piece_model_path)
        
        # Create square grid (Phase 3)
        self.square_grid = self._create_square_grid()
        
        print("✅ Chess Board System initialized")
    
    def lock_orientation(self, rotation: int):
        """Lock the board rotation (0, 90, 180, 270) to prevent flipping."""
        print(f"🔒 Locking board orientation to {rotation}°")
        self.locked_rotation = rotation
        
    def unlock_orientation(self):
        self.locked_rotation = None

    def _create_square_grid(self) -> Dict:
        """
        Phase 3: Create 8x8 square grid mapping with margins
        
        Returns: dict mapping square names (e.g., 'e4') to pixel coordinates
        """
        # Calculate usable board area (excluding margins)
        usable_size = self.board_size - (2 * self.grid_margin)
        square_size = usable_size / 8
        
        grid = {}
        
        files = 'abcdefgh'  # Columns (a-h)
        ranks = '12345678'  # Rows (1-8)
        
        for rank_idx, rank in enumerate(ranks):
            for file_idx, file_letter in enumerate(files):
                square_name = f"{file_letter}{rank}"
                
                # Calculate pixel coordinates with margin offset
                # Rank 1 is at bottom (y = 7*square_size + margin)
                x1 = self.grid_margin + (file_idx * square_size)
                y1 = self.grid_margin + ((7 - rank_idx) * square_size)  # Flip Y
                x2 = x1 + square_size
                y2 = y1 + square_size
                
                grid[square_name] = {
                    'bbox': (int(x1), int(y1), int(x2), int(y2)),
                    'center': (int((x1 + x2) / 2), int((y1 + y2) / 2)),
                    'file': file_letter,
                    'rank': rank
                }
        
        return grid
    
    def calibrate_grid_margin(
        self,
        board_image: np.ndarray,
        pieces: List[PieceDetection],
        transform_matrix: np.ndarray,
        frame_shape: Tuple[int, int],
        test_margins: List[int] = [0, 10, 15, 20, 25, 30, 40]
    ) -> int:
        """
        Auto-calibrate grid margin by finding best fit to piece positions
        
        Args:
            board_image: Warped board image
            pieces: Detected pieces
            transform_matrix: Perspective transform matrix
            frame_shape: Original frame shape
            test_margins: List of margin values to test
        
        Returns:
            Optimal margin value
        """
        if len(pieces) < 8:
            print("⚠️  Not enough pieces for calibration (need at least 8)")
            return self.grid_margin
        
        best_margin = self.grid_margin
        best_score = float('inf')
        
        h, w = frame_shape
        
        print(f"\n🔧 Calibrating grid margin (testing {len(test_margins)} values)...")
        
        for margin in test_margins:
            # Temporarily create grid with this margin
            temp_usable = self.board_size - (2 * margin)
            temp_square_size = temp_usable / 8
            
            # Calculate distances from pieces to square centers
            distances = []
            
            for piece in pieces:
                # Transform piece to board coordinates
                x_px = piece.x_center * w
                y_px = piece.y_center * h
                point = np.array([[[x_px, y_px]]], dtype=np.float32)
                transformed = cv2.perspectiveTransform(point, transform_matrix)[0][0]
                
                # Find nearest square with this margin
                min_dist = float('inf')
                
                for rank_idx in range(8):
                    for file_idx in range(8):
                        x_center = margin + (file_idx + 0.5) * temp_square_size
                        y_center = margin + ((7 - rank_idx) + 0.5) * temp_square_size
                        
                        dist = np.linalg.norm(transformed - [x_center, y_center])
                        if dist < min_dist:
                            min_dist = dist
                
                distances.append(min_dist)
            
            # Score = average distance (lower is better)
            avg_distance = np.mean(distances)
            
            print(f"   Margin {margin:2d}px: avg distance = {avg_distance:.2f}px")
            
            if avg_distance < best_score:
                best_score = avg_distance
                best_margin = margin
        
        print(f"\n✅ Optimal margin: {best_margin}px (avg distance: {best_score:.2f}px)")
        return best_margin
    
    def set_grid_margin(self, margin: int):
        """
        Update grid margin and regenerate square grid
        
        Args:
            margin: New margin value in pixels
        """
        self.grid_margin = margin
        self.square_grid = self._create_square_grid()
        print(f"✅ Grid margin updated to {margin}px")
    
    def detect_corners(self, frame: np.ndarray, conf: float = 0.01) -> Optional[np.ndarray]:
        """
        Detect 4 board corners using corner detection model
        
        Returns: Array of 4 corner points or None
        """
        # Run detection
        results = self.corner_model(frame, conf=conf, verbose=False)
        
        # Parse detections
        corners = []
        for result in results:
            for box in result.boxes:
                x, y = box.xywhn[0][:2].cpu().numpy()
                confidence = float(box.conf[0])
                corners.append({'x': x, 'y': y, 'conf': confidence})
        
        if len(corners) < 4:
            return None
        
        # Select best corner from each quadrant
        quadrants = {'tl': [], 'tr': [], 'br': [], 'bl': []}
        
        for c in corners:
            if c['x'] < 0.5 and c['y'] < 0.5:
                quadrants['tl'].append(c)
            elif c['x'] >= 0.5 and c['y'] < 0.5:
                quadrants['tr'].append(c)
            elif c['x'] >= 0.5 and c['y'] >= 0.5:
                quadrants['br'].append(c)
            else:
                quadrants['bl'].append(c)
        
        # Get best from each quadrant
        board_corners = []
        for quad in ['tl', 'tr', 'br', 'bl']:
            if not quadrants[quad]:
                return None
            best = max(quadrants[quad], key=lambda c: c['conf'])
            board_corners.append([best['x'], best['y']])
        
        return np.float32(board_corners)
    
    def detect_pieces(self, frame: np.ndarray, conf: float = 0.3) -> List[PieceDetection]:
        """
        Detect chess pieces using SAHI or standard inference
        
        Returns: List of PieceDetection objects
        """
        pieces = []
        
        if self.use_sahi:
            # SAHI sliced inference
            result = get_sliced_prediction(
                frame,
                self.piece_model,
                slice_height=self.slice_size,
                slice_width=self.slice_size,
                overlap_height_ratio=self.overlap_ratio,
                overlap_width_ratio=self.overlap_ratio
            )
            
            h, w = frame.shape[:2]
            for obj in result.object_prediction_list:
                bbox = obj.bbox
                x_center = (bbox.minx + bbox.maxx) / 2 / w
                y_center = (bbox.miny + bbox.maxy) / 2 / h
                width = (bbox.maxx - bbox.minx) / w
                height = (bbox.maxy - bbox.miny) / h
                
                pieces.append(PieceDetection(
                    class_id=obj.category.id,
                    class_name=obj.category.name,
                    confidence=obj.score.value,
                    x_center=x_center,
                    y_center=y_center,
                    width=width,
                    height=height
                ))
        else:
            # Standard YOLO inference
            results = self.piece_model(frame, conf=conf, verbose=False)
            
            for result in results:
                for box in result.boxes:
                    cls_id = int(box.cls[0])
                    confidence = float(box.conf[0])
                    x, y, w, h = box.xywhn[0].cpu().numpy()
                    
                    pieces.append(PieceDetection(
                        class_id=cls_id,
                        class_name=result.names[cls_id],
                        confidence=confidence,
                        x_center=float(x),
                        y_center=float(y),
                        width=float(w),
                        height=float(h)
                    ))
        
        return pieces
    
    def detect_orientation(self, pieces: List[PieceDetection]) -> Tuple[int, Dict]:
        """
        Detect board orientation by analyzing piece positions
        
        Returns:
            rotation: 0, 90, 180, or 270 degrees
            metadata: Additional orientation info
        """
        if self.locked_rotation is not None:
            return self.locked_rotation, {'confidence': 'locked', 'rotation': self.locked_rotation}
            
        if not pieces:
            return 0, {'confidence': 'none', 'reason': 'no_pieces'}
        
        # Separate by color
        white_pieces = [p for p in pieces if p.color == 'white']
        black_pieces = [p for p in pieces if p.color == 'black']
        
        if not white_pieces or not black_pieces:
            return 0, {'confidence': 'low', 'reason': 'missing_colors'}
        
        # Calculate average Y positions
        white_y_avg = np.mean([d.y_center for d in white_pieces])
        black_y_avg = np.mean([d.y_center for d in black_pieces])
        
        # Use pawns for higher confidence if available
        white_pawns = [d for d in white_pieces if 'pawn' in d.piece_type.lower()]
        black_pawns = [d for d in black_pieces if 'pawn' in d.piece_type.lower()]
        
        confidence = 'medium'
        
        if white_pawns and black_pawns:
            white_pawn_y = np.mean([d.y_center for d in white_pawns])
            black_pawn_y = np.mean([d.y_center for d in black_pawns])
            confidence = 'high'
        else:
            white_pawn_y = white_y_avg
            black_pawn_y = black_y_avg
        
        # Determine rotation
        y_diff = white_pawn_y - black_pawn_y
        
        if abs(y_diff) < 0.1:
            # Pieces at similar Y -> board rotated 90° or 270°
            white_x_avg = np.mean([d.x_center for d in white_pieces])
            black_x_avg = np.mean([d.x_center for d in black_pieces])
            
            # INVERTED LOGIC based on user feedback
            # Previously: 270 if white > black else 90
            # New: 90 if white > black else 270
            rotation = 90 if white_x_avg > black_x_avg else 270
            confidence = 'medium'
        else:
            # Normal or 180° rotation
            rotation = 0 if y_diff > 0 else 180
        
        metadata = {
            'rotation': rotation,
            'confidence': confidence,
            'white_avg_y': float(white_y_avg),
            'black_avg_y': float(black_y_avg),
            'y_difference': float(y_diff),
            'white_count': len(white_pieces),
            'black_count': len(black_pieces)
        }
        
        return rotation, metadata
    
    def apply_perspective_transform(
        self,
        frame: np.ndarray,
        corners: np.ndarray,
        rotation: int = 0
    ) -> Tuple[np.ndarray, np.ndarray]:
        """
        Apply perspective transform to get top-down board view
        
        Args:
            frame: Input frame
            corners: 4 corner points (top-left, top-right, bottom-right, bottom-left of IMAGE)
            rotation: Board rotation (0, 90, 180, 270) to orient a1 at bottom-left
        
        Returns: (warped_board, transform_matrix)
        """
        h, w = frame.shape[:2]
        
        # Convert normalized corners to pixels
        if corners.max() <= 1.0:
            corners_px = corners.copy()
            corners_px[:, 0] *= w
            corners_px[:, 1] *= h
        else:
            corners_px = corners
        
        # Destination points (square board with a1 at bottom-left)
        dst = np.float32([
            [0, 0],                          # a8 (top-left)
            [self.board_size, 0],            # h8 (top-right)
            [self.board_size, self.board_size],  # h1 (bottom-right)
            [0, self.board_size]             # a1 (bottom-left)
        ])
        
        # Rotate destination based on detected orientation
        if rotation == 90:
            # Rotate CCW: map corners so a1 ends up at bottom-left
            dst = np.float32([
                [self.board_size, 0],            # h8 -> a8
                [self.board_size, self.board_size],  # h1 -> h8
                [0, self.board_size],            # a1 -> h1
                [0, 0]                           # a8 -> a1
            ])
        elif rotation == 180:
            # Rotate 180°
            dst = np.float32([
                [self.board_size, self.board_size],  # h1 -> a8
                [0, self.board_size],            # a1 -> h8
                [0, 0],                          # a8 -> h1
                [self.board_size, 0]             # h8 -> a1
            ])
        elif rotation == 270:
            # Rotate CW
            dst = np.float32([
                [0, self.board_size],            # a1 -> a8
                [0, 0],                          # a8 -> h8
                [self.board_size, 0],            # h8 -> h1
                [self.board_size, self.board_size]   # h1 -> a1
            ])
        
        # Get transform matrix
        matrix = cv2.getPerspectiveTransform(corners_px, dst)
        
        # Warp frame
        warped = cv2.warpPerspective(frame, matrix, (self.board_size, self.board_size))
        
        return warped, matrix
    
    def map_pieces_to_squares(
        self,
        pieces: List[PieceDetection],
        transform_matrix: np.ndarray,
        frame_shape: Tuple[int, int],
        rotation: int = 0
    ) -> Dict[str, Tuple[str, str]]:
        """
        Phase 3: Map detected pieces to chess squares
        """
        board_state = {}
        h, w = frame_shape
        offset_factor = 0.25 # Shift towards feet (0.3 = 80% down)
        
        for piece in pieces:
            # Calculate feet position based on rotation
            # x_center, y_center are normalized (0-1)
            
            if rotation == 180: 
                # Black at bottom, pieces pointing UP (Head Up) -> Feet at Bottom (+Y)
                feat_x = piece.x_center
                feat_y = piece.y_center + (piece.height * offset_factor)
            elif rotation == 0:
                # White at bottom, pieces pointing DOWN (Head Down) -> Feet at Top (-Y)
                feat_x = piece.x_center
                feat_y = piece.y_center - (piece.height * offset_factor)
            elif rotation == 270:
                # Pieces pointing LEFT (Head Left) -> Feet at Right (+X)
                feat_x = piece.x_center + (piece.width * offset_factor)
                feat_y = piece.y_center
            elif rotation == 90:
                # Pieces pointing RIGHT (Head Right) -> Feet at Left (-X)
                feat_x = piece.x_center - (piece.width * offset_factor)
                feat_y = piece.y_center
            else:
                # Default to Bottom (+Y)
                feat_x = piece.x_center
                feat_y = piece.y_center + (piece.height * offset_factor)
            
            x_px = feat_x * w
            y_px = feat_y * h
            
            # print(f"Piece {piece.piece_type} mapped at ({x_px:.1f}, {y_px:.1f}) Rot:{rotation}")
            
            # Transform to board coordinates
            point = np.array([[[x_px, y_px]]], dtype=np.float32)
            transformed = cv2.perspectiveTransform(point, transform_matrix)[0][0]
            
            # Find closest square
            min_dist = float('inf')
            closest_square = None
            
            for square_name, square_data in self.square_grid.items():
                center = square_data['center']
                dist = np.linalg.norm(transformed - center)
                
                if dist < min_dist:
                    min_dist = dist
                    closest_square = square_name
            
            # Only assign if within reasonable distance (avoid outliers)
            if closest_square and min_dist < (self.board_size // 8):
                board_state[closest_square] = (piece.piece_type, piece.color)
        
        return board_state
    
    def initialize_board_detection(
        self,
        frame: np.ndarray,
        corner_conf: float = 0.01
    ) -> Optional[Tuple]:
        """
        Detect corners once at the start of video processing.
        Returns corners for reuse throughout the video.
        
        Returns: (corners, initial_rotation) or None if failed
        """
        print("🔍 Initializing board detection (one-time corner detection)...")
        corners = self.detect_corners(frame, corner_conf)
        if corners is None:
            print("❌ Failed to detect corners in initial frame")
            return None
        
        # Detect initial pieces to determine rotation
        pieces = self.detect_pieces(frame, 0.3)
        rotation, _ = self.detect_orientation(pieces)
        
        print(f"✅ Board initialized: {len(corners)} corners detected, rotation={rotation}°")
        return corners, rotation
    
    def process_frame(
        self,
        frame: np.ndarray,
        corner_conf: float = 0.01,
        piece_conf: float = 0.3,
        cached_corners: Optional[List] = None,
        cached_rotation: Optional[int] = None
    ) -> Optional[Dict]:
        """
        Complete frame processing: corners, pieces, orientation, squares, board state
        
        Args:
            cached_corners: Pre-detected corners from initialization (optional)
            cached_rotation: Pre-detected rotation from initialization (optional)
        
        Returns: Dict with all detection results or None if failed
        """
        # 1. Use cached corners if provided, otherwise detect
        if cached_corners is not None:
            corners = cached_corners
        else:
            corners = self.detect_corners(frame, corner_conf)
            if corners is None:
                return None
        
        # 2. Detect pieces
        pieces = self.detect_pieces(frame, piece_conf)
        
        # 3. Use cached rotation if provided, otherwise detect
        if cached_rotation is not None:
            rotation = cached_rotation
            orientation_metadata = {'rotation': rotation, 'method': 'cached'}
        else:
            rotation, orientation_metadata = self.detect_orientation(pieces)
        
        # 4. Apply perspective transform with rotation
        board_image, transform_matrix = self.apply_perspective_transform(
            frame, corners, rotation
        )
        
        # 5. Map pieces to squares (Phase 3)
        board_state = self.map_pieces_to_squares(
            pieces,
            transform_matrix,
            frame.shape[:2],
            rotation=rotation
        )
        
        return {
            'corners': corners,
            'pieces': pieces,
            'piece_count': len(pieces),
            'rotation': rotation,
            'orientation': orientation_metadata,
            'board_image': board_image,
            'transform_matrix': transform_matrix,
            'board_state': board_state,  # Phase 3 output
            'square_grid': self.square_grid
        }
    
    def visualize_board_state(
        self,
        board_image: np.ndarray,
        board_state: Dict[str, Tuple[str, str]],
        rotation: int = 0
    ) -> np.ndarray:
        """
        Visualize board state with piece positions on squares
        
        Args:
            board_image: Top-down board image
            board_state: Piece positions
            rotation: Detected rotation (0, 90, 180, 270)
        """
        vis = board_image.copy()
        square_size = self.board_size // 8
        
        # Draw rotation indicator
        rotation_text = f"Rotation: {rotation}\u00b0"
        cv2.putText(vis, rotation_text, (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        
        # Draw grid
        for i in range(9):
            cv2.line(vis, (i * square_size, 0), (i * square_size, self.board_size), (100, 100, 100), 1)
            cv2.line(vis, (0, i * square_size), (self.board_size, i * square_size), (100, 100, 100), 1)
        
        # Draw square labels and pieces
        for square_name, square_data in self.square_grid.items():
            x, y = square_data['center']
            
            # Draw square name
            cv2.putText(vis, square_name, (x - 15, y + 5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.3, (200, 200, 200), 1)
            
            # Draw piece if present
            if square_name in board_state:
                piece_type, color = board_state[square_name]
                piece_color = (255, 255, 255) if color == 'white' else (0, 0, 0)
                
                # Draw circle for piece
                cv2.circle(vis, (x, y), 20, piece_color, -1)
                cv2.circle(vis, (x, y), 22, (0, 255, 0), 2)
                
                # Draw piece type abbreviation
                abbrev = piece_type[0].upper()
                cv2.putText(vis, abbrev, (x - 5, y + 5),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        
        return vis
    
    def export_board_state(self, board_state: Dict[str, Tuple[str, str]]) -> str:
        """
        Export board state as JSON
        Note: Tuples will be converted to lists in JSON
        """
        return json.dumps(board_state, indent=2)
    
    @staticmethod
    def load_board_state(json_str: str) -> Dict[str, Tuple[str, str]]:
        """
        Load board state from JSON string
        Converts lists back to tuples
        
        Args:
            json_str: JSON string from export_board_state()
        
        Returns:
            Board state dict with tuples
        """
        loaded = json.loads(json_str)
        return {k: tuple(v) for k, v in loaded.items()}


# ============================================================================
# MoveTracker and ChessLogicValidator Classes
# ============================================================================
"""
Move Tracker with Stability Buffer and Chess Logic Validation

Uses the "stability buffer" approach: only trust a board state after it's been
stable for N consecutive frames. This filters out hand occlusion naturally.

Also validates moves against chess rules using python-chess, with visual override
for resynchronization when the engine falls out of sync.
"""

import json
import chess
from collections import Counter
from typing import Dict, Tuple, Optional, List
from dataclasses import dataclass


@dataclass
class DetectedMove:
    """Detected chess move with metadata"""
    from_square: str
    to_square: str
    piece_type: str
    piece_color: str
    captured_piece: Optional[Tuple[str, str]] = None
    is_castling: bool = False
    is_promotion: bool = False
    promotion_piece: Optional[str] = None
    uci: str = ""
    san: str = ""
    confidence: str = "visual"  # "visual", "validated", "forced"
    
    def __str__(self):
        if self.is_castling:
            return "O-O" if self.to_square in ['g1', 'g8'] else "O-O-O"
        
        notation = f"{self.from_square}-{self.to_square}"
        if self.captured_piece:
            notation += f" (captures {self.captured_piece[1]} {self.captured_piece[0]})"
        if self.is_promotion:
            notation += f"={self.promotion_piece}"
        return notation


class MoveTracker:
    """
    Tracks moves using stability buffer method.
    Only registers a move when board state has been stable for N consecutive frames.
    """
    
    def __init__(self, streak_threshold: int = 7, resync_threshold: float = 3.0):
        """
        Args:
            streak_threshold: Number of consecutive frames needed to trust a state
            resync_threshold: Time in seconds before forcing visual override
        """
        self.streak_threshold = streak_threshold
        self.resync_threshold = resync_threshold
        
        # The last confirmed stable board state
        self.last_stable_state = {}
        
        # The candidate state we're currently watching
        self.candidate_state = None
        self.candidate_dict = None
        self.streak_count = 0
        
        # Track time for visual override
        self.candidate_start_time = None
        self.last_stable_time = None
        
        # Move history
        self.moves = []
        
    def update(self, current_board_state: Dict, timestamp: float) -> Optional[DetectedMove]:
        """
        Call this every frame with the board state from map_pieces_to_squares
        
        Args:
            current_board_state: Dict of {square: (piece_type, color)}
            timestamp: Current timestamp in seconds
            
        Returns:
            DetectedMove if a move was detected, None otherwise
        """
        # 1. Sanitize: Reject impossible boards immediately
        if not self._is_physically_possible(current_board_state):
            self.streak_count = 0
            self.candidate_state = None
            self.candidate_dict = None
            self.candidate_start_time = None
            return None
        
        # 2. Serialize for comparison (sort keys for consistency)
        current_hash = json.dumps(current_board_state, sort_keys=True)
        
        # 3. The Streak Check
        if current_hash == self.candidate_state:
            # State hasn't changed - increment streak
            self.streak_count += 1
        else:
            # State changed - reset streak
            self.candidate_state = current_hash
            self.candidate_dict = current_board_state
            self.streak_count = 1
            self.candidate_start_time = timestamp
        
        # 4. Trigger: Have we reached the threshold?
        if self.streak_count == self.streak_threshold:
            
            # 5. Is this actually a NEW position?
            last_hash = json.dumps(self.last_stable_state, sort_keys=True)
            
            if current_hash != last_hash and self.last_stable_state:
                # WE HAVE A MOVE!
                move_data = self._calculate_move(
                    self.last_stable_state, 
                    current_board_state,
                    timestamp
                )
                
                if move_data:
                    # Update the "official" board
                    self.last_stable_state = current_board_state.copy()
                    self.last_stable_time = timestamp
                    self.moves.append(move_data)
                    
                    return move_data
            
            # First stable state (game start)
            elif not self.last_stable_state:
                self.last_stable_state = current_board_state.copy()
                self.last_stable_time = timestamp
        
        return None
    
    def _is_physically_possible(self, state: Dict) -> bool:
        """
        Filters out frames where detection failed badly (hand blocking view).
        
        Rules:
        - Must have pieces detected
        - Cannot have more than 32 pieces
        - Should have at least 20 pieces (allows for some captures)
        - Ideally has 2 kings (one of each color)
        """
        if not state:
            return False
        
        # Count pieces by type and color
        piece_counts = Counter([v for v in state.values()])
        
        # Total pieces
        total = len(state)
        if total < 15 or total > 32:  # Too few or too many
            return False
        
        # Check kings (be lenient - occlusion can hide a king temporarily)
        white_kings = sum(1 for v in state.values() if v == ('king', 'white'))
        black_kings = sum(1 for v in state.values() if v == ('king', 'black'))
        
        # Require at least both kings detected (strict for stability)
        if white_kings != 1 or black_kings != 1:
            return False
        
        return True
    
    def _calculate_move(
        self, 
        old_state: Dict, 
        new_state: Dict,
        timestamp: float
    ) -> Optional[DetectedMove]:
        """
        Compares two board states to determine the move.
        
        Handles:
        - Normal moves (e2 → e4)
        - Captures (e4 → d5, capturing piece)
        - Castling (king + rook move together)
        """
        old_squares = set(old_state.keys())
        new_squares = set(new_state.keys())
        
        # Squares that became empty (sources)
        disappeared = old_squares - new_squares
        # Squares that became occupied (destinations)
        appeared = new_squares - old_squares
        
        # Scenario A: Normal Move (1 piece moves, no capture)
        # 1 square lost, 1 square gained
        if len(disappeared) == 1 and len(appeared) == 1:
            from_sq = list(disappeared)[0]
            to_sq = list(appeared)[0]
            piece_type, piece_color = old_state[from_sq]
            
            # CRITICAL FIX: Check piece identity
            # If piece type changed, it's likely a detection error (teleporting piece)
            # Exception: Pawn promotion
            new_type, new_color = new_state[to_sq]
            
            # Check for promotion (Pawn reaches last rank)
            is_promotion = False
            promotion_piece = None
            if piece_type == 'pawn':
                rank = int(to_sq[1])
                if (piece_color == 'white' and rank == 8) or \
                   (piece_color == 'black' and rank == 1):
                    is_promotion = True
                    promotion_piece = new_type  # The new piece type is the promotion
            
            if not is_promotion:
                # Standard move: Piece must be same type and color
                if piece_type != new_type or piece_color != new_color:
                    print(f"⚠️ Rejected 'teleport' move: {piece_color} {piece_type} at {from_sq} -> {new_color} {new_type} at {to_sq}")
                    return None
            
            return DetectedMove(
                from_square=from_sq,
                to_square=to_sq,
                piece_type=piece_type,
                piece_color=piece_color,
                is_promotion=is_promotion,
                promotion_piece=promotion_piece,
                uci=f"{from_sq}{to_sq}" + (promotion_piece[0].lower() if is_promotion and promotion_piece else "")
            )
        
        # Scenario B: Capture (1 piece moves to occupied square)
        # 1 square lost, 0 gained (destination was already occupied)
        elif len(disappeared) == 1 and len(appeared) == 0:
            from_sq = list(disappeared)[0]
            piece_type, piece_color = old_state[from_sq]
            
            # Find which square changed ownership
            capture_sq = None
            for sq in new_state:
                if sq in old_state and old_state[sq] != new_state[sq]:
                    # Piece color/type changed - this is the capture square
                    # Verify the NEW piece matches the moving piece
                    new_type, new_color = new_state[sq]
                    
                    # Check for promotion capture
                    is_promotion = False
                    if piece_type == 'pawn':
                        rank = int(sq[1])
                        if (piece_color == 'white' and rank == 8) or \
                           (piece_color == 'black' and rank == 1):
                            is_promotion = True
                    
                    if not is_promotion:
                        if new_type == piece_type and new_color == piece_color:
                            capture_sq = sq
                            break
                    else:
                        # Promotion capture: just need color match really
                        if new_color == piece_color:
                            capture_sq = sq
                            break
            
            if capture_sq:
                captured = old_state[capture_sq]
                
                # Check promotion again to get piece type
                is_promotion = False
                promotion_piece = None
                if piece_type == 'pawn':
                    rank = int(capture_sq[1])
                    if (piece_color == 'white' and rank == 8) or \
                       (piece_color == 'black' and rank == 1):
                        is_promotion = True
                        promotion_piece = new_state[capture_sq][0]

                return DetectedMove(
                    from_square=from_sq,
                    to_square=capture_sq,
                    piece_type=piece_type,
                    piece_color=piece_color,
                    captured_piece=captured,
                    is_promotion=is_promotion,
                    promotion_piece=promotion_piece,
                    uci=f"{from_sq}{capture_sq}" + (promotion_piece[0].lower() if is_promotion and promotion_piece else "")
                )
        
        # Scenario C: Castling (2 pieces move: king and rook)
        # 2 squares lost, 2 squares gained
        elif len(disappeared) == 2 and len(appeared) == 2:
            # Check if this is castling
            disappeared_list = list(disappeared)
            appeared_list = list(appeared)
            
            # Find king move
            king_moves = []
            for from_sq in disappeared_list:
                if old_state[from_sq][0] == 'king':
                    for to_sq in appeared_list:
                        if new_state[to_sq][0] == 'king':
                            king_moves.append((from_sq, to_sq, old_state[from_sq]))
            
            if king_moves:
                from_sq, to_sq, (piece_type, piece_color) = king_moves[0]
                
                # Verify this looks like castling (king moves 2 squares)
                from_file = ord(from_sq[0]) - ord('a')
                to_file = ord(to_sq[0]) - ord('a')
                
                if abs(to_file - from_file) == 2:
                    return DetectedMove(
                        from_square=from_sq,
                        to_square=to_sq,
                        piece_type='king',
                        piece_color=piece_color,
                        is_castling=True,
                        uci=f"{from_sq}{to_sq}"
                    )
        
        # Scenario D: Capture with different piece count
        # More complex scenarios - try to find the most likely move
        elif len(disappeared) > 0 and len(appeared) > 0:
            # Just take the first disappeared/appeared as the move
            # This handles edge cases where detection is slightly off
            from_sq = list(disappeared)[0]
            to_sq = list(appeared)[0]
            piece_type, piece_color = old_state[from_sq]
            
            return DetectedMove(
                from_square=from_sq,
                to_square=to_sq,
                piece_type=piece_type,
                piece_color=piece_color,
                uci=f"{from_sq}{to_sq}"
            )
        
        return None


class ChessLogicValidator:
    """
    Validates moves against chess rules using python-chess.
    Includes visual override for resynchronization.
    """
    
    def __init__(self):
        self.board = chess.Board()  # Starts at standard setup
        self.move_count = 0
        self.rejected_moves = []
        
    def validate_move(self, move: DetectedMove, visual_board_state: Dict) -> Tuple[bool, str]:
        """
        Validates a move against chess rules.
        
        Args:
            move: The detected move
            visual_board_state: Current visual board state
            
        Returns:
            (is_valid, reason)
        """
        try:
            # Try to parse the move
            chess_move = chess.Move.from_uci(move.uci)
        except ValueError:
            return False, f"Invalid UCI format: {move.uci}"
        
        # Check if move is legal
        if chess_move in self.board.legal_moves:
            # Generate SAN *before* pushing the move
            try:
                san = self.board.san(chess_move)
            except:
                san = str(move.uci)
                
            # Legal move - execute it
            self.board.push(chess_move)
            self.move_count += 1
            
            # Update move with SAN notation
            move.san = san
            move.confidence = "validated"
            
            return True, "Legal move"
        else:
            # Illegal move detected
            reason = f"Illegal: {move.uci} not in legal moves for current position"
            self.rejected_moves.append((move, reason))
            return False, reason
    
    def force_resync(self, visual_board_state: Dict) -> bool:
        """
        Forces the chess engine to match the visual board state.
        Used when visual is confident but logic says illegal (missed move).
        
        Args:
            visual_board_state: Dict of {square: (piece_type, color)}
            
        Returns:
            True if resync successful
        """
        try:
            fen = self._visual_state_to_fen(visual_board_state)
            self.board.set_fen(fen)
            print(f"🔄 RESYNCED engine to FEN: {fen}")
            return True
        except Exception as e:
            print(f"❌ Resync failed: {e}")
            return False
    
    def _visual_state_to_fen(self, state: Dict) -> str:
        """
        Convert visual board state dict to FEN notation.
        
        FEN format: piece placement / active color / castling / en passant / halfmove / fullmove
        """
        # Create 8x8 board representation
        board_array = [[None for _ in range(8)] for _ in range(8)]
        
        # Map pieces to board
        piece_symbols = {
            ('pawn', 'white'): 'P',
            ('knight', 'white'): 'N',
            ('bishop', 'white'): 'B',
            ('rook', 'white'): 'R',
            ('queen', 'white'): 'Q',
            ('king', 'white'): 'K',
            ('pawn', 'black'): 'p',
            ('knight', 'black'): 'n',
            ('bishop', 'black'): 'b',
            ('rook', 'black'): 'r',
            ('queen', 'black'): 'q',
            ('king', 'black'): 'k',
        }
        
        for square, piece_info in state.items():
            file = ord(square[0]) - ord('a')  # 0-7
            rank = int(square[1]) - 1  # 0-7
            symbol = piece_symbols.get(piece_info, '?')
            board_array[7 - rank][file] = symbol  # FEN starts from rank 8
        
        # Convert to FEN piece placement
        fen_rows = []
        for row in board_array:
            fen_row = ""
            empty_count = 0
            for cell in row:
                if cell is None:
                    empty_count += 1
                else:
                    if empty_count > 0:
                        fen_row += str(empty_count)
                        empty_count = 0
                    fen_row += cell
            if empty_count > 0:
                fen_row += str(empty_count)
            fen_rows.append(fen_row)
        
        piece_placement = "/".join(fen_rows)
        
        # Default other FEN fields (we don't track these visually)
        # Active color: guess based on move count
        active_color = "w" if self.move_count % 2 == 0 else "b"
        castling = "KQkq"  # Assume all castling available
        en_passant = "-"
        halfmove = "0"
        fullmove = str((self.move_count // 2) + 1)
        
        fen = f"{piece_placement} {active_color} {castling} {en_passant} {halfmove} {fullmove}"
        return fen
    
    def get_fen(self) -> str:
        """Get current FEN position"""
        return self.board.fen()
    
    def get_legal_moves(self) -> List[str]:
        """Get all legal moves in UCI format"""
        return [move.uci() for move in self.board.legal_moves]




In [ ]:
# ============================================================================
# MoveDetectorV4 Class
# ============================================================================
"""
Phase 4: Move Detection V4 - Window Voting Stability
Tracks board state using a sliding window voting mechanism to handle detection flicker.

Algorithm:
1. Buffer N frames of board states.
2. Find the "Consensus State" (most frequent state in buffer).
3. If Consensus frequency > Confidence Threshold:
   - Mark as "Sure State".
4. Compare current Sure State vs Previous Sure State to find moves.
"""

import cv2
import numpy as np
import json
from collections import Counter
from typing import List, Dict, Tuple, Optional, Deque
from collections import deque

class MoveDetectorV4:
    def __init__(
        self,
        system: ChessBoardSystem,
        window_size: int = 25,       # Look at last 15 frames
        confidence_threshold: float = 0.9, # Require 70% agreement (11/15 frames)
        use_validation: bool = True
    ):
        self.system = system
        self.window_size = window_size
        self.confidence_threshold = confidence_threshold
        self.use_validation = use_validation
        
        # Buffer for sliding window
        self.state_buffer: Deque[str] = deque(maxlen=window_size)
        self.raw_states: Deque[Dict] = deque(maxlen=window_size) # Keep raw dicts to retrieve later
        
        # "Sure States" history
        self.sure_states: List[Dict] = []
        self.detected_moves: List[DetectedMove] = []
        
        # Validator
        self.validator = ChessLogicValidator()
        
        # Tracker helper (reused for move calculation logic)
        self.tracker_helper = MoveTracker() 
        
        self.current_consensus_hash = None
        
        # Track detection failures
        self.consecutive_failures = 0
        self.failure_threshold = 10  # Clear buffer after 10 failed frames

    def detect_moves(self, video_path: str, debug_video_path: Optional[str] = None) -> List[DetectedMove]:
        """Alias for process_video for compatibility"""
        return self.process_video(video_path, debug_video_path)
    
    def process_video(self, video_path: str, debug_video_path: Optional[str] = None) -> List[DetectedMove]:
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        video_writer = None
        if debug_video_path:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            video_writer = cv2.VideoWriter(debug_video_path, fourcc, fps, (width, height))
            
        frame_idx = 0
        
        print(f"\n🔎 Starting Analysis (V4 - Voting Stability)")
        print(f"   Window Size: {self.window_size}")
        print(f"   Confidence: {self.confidence_threshold*100}%")
        print("="*60)
        
        # Initialize board detection once (detect corners in first frame)
        ret, first_frame = cap.read()
        if not ret:
            print("❌ Failed to read first frame")
            return []
        
        board_init = self.system.initialize_board_detection(first_frame)
        if board_init is None:
            print("❌ Failed to initialize board detection")
            return []
        
        cached_corners, cached_rotation = board_init
        
        # Reset video to beginning
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

        while True:
            ret, frame = cap.read()
            if not ret:
                break
                
            # Process frame with cached corners and rotation
            result = self.system.process_frame(
                frame, 
                cached_corners=cached_corners,
                cached_rotation=cached_rotation
            )
            
            if result:
                # Reset failure counter on successful detection
                self.consecutive_failures = 0
                
                current_state = result['board_state']
                state_hash = json.dumps(current_state, sort_keys=True)
                
                # Add to buffer
                self.state_buffer.append(state_hash)
                self.raw_states.append(current_state)
                
                # Check for Consensus
                if len(self.state_buffer) == self.window_size:
                    consensus_hash, count = Counter(self.state_buffer).most_common(1)[0]
                    confidence = count / self.window_size
                    
                    if confidence >= self.confidence_threshold:
                        # We have a "Sure State"
                        
                        # Is it different from the CURRENT sure state?
                        if consensus_hash != self.current_consensus_hash:
                            # Retrieve the actual state dict
                            consensus_state = None
                            for i, h in enumerate(self.state_buffer):
                                if h == consensus_hash:
                                    consensus_state = self.raw_states[i]
                                    break
                            
                            # PIECE COUNT CHECK
                            # Filter out states where hand blocks many pieces
                            if self.sure_states:
                                prev_state = self.sure_states[-1]
                                prev_count = len(prev_state)
                                curr_count = len(consensus_state)
                                diff = curr_count - prev_count
                                
                                # Valid changes:
                                # 0: Normal move
                                # -1: Capture
                                # +1: Only allowed if recovering from missed detection (but suspicious)
                                # >1 or <-1: Rejected (Hand occlusion or error)
                                
                                if diff < -1 or diff > 1:
                                    # Too many pieces changed -> Likely hand occlusion
                                    # Ignore this "Sure State" until it stabilizes to correct count
                                    # print(f"   ⚠️ Ignoring state: Piece count change {diff} (likely occlusion)")
                                    pass
                                else:
                                    # Valid piece count change -> Process it
                                    self._process_new_state(consensus_state, consensus_hash, frame_idx, fps)
                            else:
                                # First state -> Always accept
                                self._process_new_state(consensus_state, consensus_hash, frame_idx, fps)
                                
                                # LOCK ORIENTATION to prevent flipping during game
                                # Once we have a stable initial state, we trust this orientation.
                                current_rotation = result.get('rotation', 0)
                                self.system.lock_orientation(current_rotation)
            else:
                # Detection failed (hand blocking corners/pieces)
                self.consecutive_failures += 1
                
                # If failures exceed threshold, clear buffer to allow new consensus
                if self.consecutive_failures >= self.failure_threshold:
                    if len(self.state_buffer) > 0:
                        print(f"\n⚠️  Detection lost for {self.consecutive_failures} frames. Clearing buffer...")
                        self.state_buffer.clear()
                        self.raw_states.clear()
                        # Reset consensus to allow fresh detection
                        # but keep sure_states history intact
            
            # Draw Debug
            if video_writer:
                vis = self._draw_debug(frame, result)
                video_writer.write(vis)
                
            if frame_idx % 30 == 0:
                progress = frame_idx / cap.get(cv2.CAP_PROP_FRAME_COUNT) * 100
                print(f"\rProcessing... {progress:.1f}%", end="")
                
            frame_idx += 1
            
        cap.release()
        if video_writer:
            video_writer.release()
            
        return self.detected_moves

    def _process_new_state(self, consensus_state, consensus_hash, frame_idx, fps):
        print(f"🔒 New Sure State found at frame {frame_idx} (Pieces: {len(consensus_state)})")
        
        if not self.sure_states:
            self.sure_states.append(consensus_state)
            self.current_consensus_hash = consensus_hash
            
            # Sync validator to initial state
            try:
                fen = self.validator._visual_state_to_fen(consensus_state)
                self.validator.board.set_fen(fen)
                print(f"   Initial FEN: {fen}")
            except:
                pass
        else:
            # Compare with previous Sure State to find Move
            prev_state = self.sure_states[-1]
            timestamp = frame_idx / fps
            
            move = self.tracker_helper._calculate_move(prev_state, consensus_state, timestamp)
            
            if move:
                print(f"🎯 Potential Move: {move}")
                
                if self.use_validation:
                    is_valid, reason = self.validator.validate_move(move, consensus_state)
                    if is_valid:
                        print(f"   ✅ Validated: {move.san}")
                        self.detected_moves.append(move)
                        self.sure_states.append(consensus_state)
                        self.current_consensus_hash = consensus_hash
                    else:
                        print(f"   ⚠️ Rejected: {reason}")
                        # If we are VERY confident in this state, force resync
                        if True: # Always resync if piece count is valid (since we filtered strict count)
                            print("   🔄 Force Resync (Valid Piece Count State)")
                            # Resync to previous state first
                            self.validator.force_resync(prev_state)
                            
                            # Now try to validate the move from the resynced position
                            try:
                                import chess
                                chess_move = chess.Move.from_uci(move.uci)
                                if chess_move in self.validator.board.legal_moves:
                                    move.san = self.validator.board.san(chess_move)
                                    self.validator.board.push(chess_move)
                                    self.validator.move_count += 1
                                    move.confidence = "validated_after_resync"
                                else:
                                    # Still not legal after resync, force to new state
                                    self.validator.force_resync(consensus_state)
                                    # Generate basic SAN from UCI
                                    move.san = self._uci_to_basic_san(move)
                                    move.confidence = "forced"
                            except:
                                # Fallback: generate basic SAN
                                move.san = self._uci_to_basic_san(move)
                                move.confidence = "forced"
                                self.validator.force_resync(consensus_state)
                            
                            self.detected_moves.append(move)
                            self.sure_states.append(consensus_state)
                            self.current_consensus_hash = consensus_hash
                else:
                    self.detected_moves.append(move)
                    self.sure_states.append(consensus_state)
                    self.current_consensus_hash = consensus_hash
            
    def _uci_to_basic_san(self, move) -> str:
        """
        Generate basic SAN notation from move info when chess engine can't validate.
        This is a fallback for forced moves.
        Includes check (+) and checkmate (#) detection.
        """
        import chess
        
        # Piece symbols (pawn has no symbol)
        piece_symbols = {
            'king': 'K',
            'queen': 'Q',
            'rook': 'R',
            'bishop': 'B',
            'knight': 'N',
            'pawn': ''
        }
        
        piece_symbol = piece_symbols.get(move.piece_type, '')
        
        # Handle castling
        if move.is_castling:
            notation = "O-O" if move.to_square[0] == 'g' else "O-O-O"
        else:
            # Build notation
            notation = piece_symbol
            
            # For captures, add 'x'
            if move.captured_piece:
                # Pawns need the file letter before 'x'
                if not piece_symbol:  # Pawn
                    notation += move.from_square[0]  # File (e.g., 'e')
                notation += 'x'
            
            # Destination square
            notation += move.to_square
            
            # Handle promotion
            if move.is_promotion and move.promotion_piece:
                promo_symbol = piece_symbols.get(move.promotion_piece, 'Q')
                notation += f'={promo_symbol}'
        
        # Add check/checkmate symbols
        # NOTE: For forced moves, the validator's board has already been resynced to the new state
        # So we check the CURRENT board state, not replay the move
        try:
            if hasattr(self, 'validator') and self.validator:
                # Check if current position (after move) has opponent in check
                if self.validator.board.is_checkmate():
                    notation += '#'
                elif self.validator.board.is_check():
                    notation += '+'
        except:
            # If check detection fails, just return notation without check symbols
            pass
        
        return notation
    
    def _draw_debug(self, frame, result):
        # Reuse V2 visualization logic for consistency, but simplified
        if result:
            from .move_detector_v2 import MoveDetectorV2
            # Hacky: create temp v2 to use its draw method
            # Ideally we refactor visualization to a shared util
            v2 = MoveDetectorV2(self.system)
            streak = int((Counter(self.state_buffer).most_common(1)[0][1] / self.window_size) * 10) if self.state_buffer else 0
            return v2._draw_debug_frame(frame, result, streak, 10, self.detected_moves[-1] if self.detected_moves else None)
        return frame

In [ ]:
# ============================================================================
# Helper Functions
# ============================================================================
def moves_to_pgn(moves: List[DetectedMove]) -> str:
    """
    Convert detected moves to PGN format with move pairs.
    
    Format:
    - White starts: 1. e4 e5 2. Nf3 Nc6
    - Black starts: 1. f4 2. Ng6 fxg3 3. Nxh8
    """
    if not moves:
        return ""
    
    pgn_parts = []
    move_number = 1
    
    # Determine if first move is white or black
    first_is_white = (moves[0].piece_color == 'white')
    
    i = 0
    while i < len(moves):
        if i == 0 and not first_is_white:
            # Black moves first - just the move, no pairing
            pgn_parts.append(f"{move_number}. {moves[i].san}")
            i += 1
            move_number += 1
        else:
            # Normal pairing: white then black
            if i < len(moves):
                white_move = moves[i].san
                i += 1
                
                # Check if there's a black response
                if i < len(moves):
                    black_move = moves[i].san
                    pgn_parts.append(f"{move_number}. {white_move} {black_move}")
                    i += 1
                else:
                    # Only white move
                    pgn_parts.append(f"{move_number}. {white_move}")
                
                move_number += 1
    
    return " ".join(pgn_parts)
def create_submission_csv(video_results: Dict[str, str], output_path: str = 'submission.csv'):
    """
    Create submission CSV in the required format.
    
    Format:
        row_id,output
        video_1.mp4,1. e4 e5 2. Nf3 Nc6
        (Bonus)Long_video_student.mp4,1. d4 d5 2. c4 c6
    """
    rows = []
    for video_name, pgn in video_results.items():
        rows.append({
            'row_id': video_name,
            'output': pgn if pgn else ""  # Ensure no NULL values
        })
    
    # Create DataFrame with exact column order
    df = pd.DataFrame(rows, columns=['row_id', 'output'])
    
    # Save CSV with header
    df.to_csv(output_path, index=False)
    
    print(f"\n💾 CSV saved to: {output_path}")
    print(f"   Total videos: {len(rows)}")
    print(f"   NULL values: {df['output'].isna().sum()} (should be 0)")
    
    return df

print("✅ All classes and functions loaded")

In [ ]:
# CELL: Main Function (DO NOT CHANGE FUNCTION NAME)
# This function will be used for marking

def chess_move(video_name):
    """
    Input: video filename (.mp4)
    Output: return string of Chess moves for video_name in PGN format
    """
    try:
        # Initialize system
        system = ChessBoardSystem(
            corner_model_path=CORNER_MODEL_PATH,
            piece_model_path=PIECE_MODEL_PATH,
            board_size=BOARD_SIZE,
            grid_margin=GRID_MARGIN,
            use_sahi=False
        )
        
        # Reset orientation for this video
        system.unlock_orientation()
        
        # Create detector
        detector = MoveDetectorV4(
            system=system,
            window_size=WINDOW_SIZE,
            confidence_threshold=CONFIDENCE_THRESHOLD,
            use_validation=True
        )
        
        # Process video
        detected_moves = detector.process_video(str(video_name), debug_video_path=None)
        
        # Convert to PGN
        moves = moves_to_pgn(detected_moves)
        
        # Ensure no NULL - return empty string if no moves
        if not moves:
            return ""
        
        return moves
        
    except Exception as e:
        print(f"Error processing {video_name}: {str(e)}")
        import traceback
        traceback.print_exc()
        return ""  # Return empty string on error (no NULL)

print("✅ chess_move function defined")


In [ ]:
# CELL 3: Marking
# TAs will use this section to check the PGN during the hidden video score evaluation.
# You can also use it for testing with the evaluation video set.

# Uncomment if needed:
!pip install editdistance
!pip install opencv-python


In [ ]:
!wget https://drive.google.com/uc?id=1a96Ei0hwfMvYUyUK-d0ZxvK6m6uSoAuR -O chess_solution_final_seen.csv

In [ ]:
# CELL 4: Download utility function
import subprocess

def download_file(url, filename=None):
    """
    Downloads a file from the specified URL using wget.
    
    Parameters:
        url (str): The URL of the file to download.
        filename (str, optional): The output filename.
    """
    cmd = ['wget', url]
    if filename:
        cmd.extend(['-O', filename])
    
    try:
        print(cmd)
        subprocess.run(cmd, check=True)
        print(f"Downloaded {url} successfully.")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")


In [ ]:
import os
import shutil

SOURCE_DIR = '/kaggle/input/chess-videos/Chess Detection Competition'
DEST_DIR = '/kaggle/working'

# Build a dictionary of all files inside SOURCE_DIR (recursive)
all_files = {}
for root, dirs, files in os.walk(SOURCE_DIR):
    for file in files:
        full_path = os.path.join(root, file)
        all_files[file] = full_path   # key = filename, value = full path

print(f"📁 Indexed {len(all_files)} files from input directory")

# Open CSV
with open('/kaggle/working/chess_solution_final_seen.csv', 'r') as f:
    _ = f.readline()  # skip header

    for line in f:
        file_name = line.strip().split(',')[0]

        if file_name in all_files:
            source_path = all_files[file_name]
            dest_path = os.path.join(DEST_DIR, file_name)

            print(f"📂 Copying: {file_name}")
            shutil.copy(source_path, dest_path)
        else:
            print(f"❌ Not found anywhere: {file_name}")

print("✅ Done copying all available videos.")

In [ ]:
# CELL 6: Scoring your output
import editdistance

#Example scoring code
#Uncomment and modify as needed



f = open('/kaggle/working/chess_solution_final_seen.csv','r')
score = 0
n = 0
for line in f:
    video_name, moves_sol, p, link = line.split(',')
    moves = chess_move(video_name)
    s = 1 - editdistance.eval(moves, moves_sol) / max(len(moves), len(moves_sol))
    score += s
    print(video_name)
    print(f"your result: {moves}")
    print(f"solution   : {moves_sol}")
    print(f"The edit distance between your results and solution is {s}.")
    n += 1
f.close()
if n != 0:
    print("-----------------------")
    print("Total score :", score/n)
    print("-----------------------")


In [ ]:
# Optional: Test with a single video
# Uncomment to test:
# test_video = "/kaggle/working/(Bonus)Long_Video_label.mp4"  # Change this to your test video
# result = chess_move(test_video)
# print(f"Video: {test_video}")
# print(f"PGN: {result}")
# print(f"Moves: {result.count('.')}")


In [ ]:
# CELL 7: Batch Process Videos and Generate CSV
# This cell processes all videos in a directory and creates the submission CSV

def process_all_videos_to_csv(video_dir: str, output_csv: str = 'submission.csv'):
    """
    Process all videos in directory and generate submission CSV.
    
    Args:
        video_dir: Directory containing video files
        output_csv: Output CSV filename
    """
    video_dir = Path(video_dir)
    
    # Get all video files
    video_files = list(video_dir.glob("*.mp4")) + list(video_dir.glob("*.mov"))
    
    # Check for bonus videos
    bonus_patterns = ["*Bonus*", "*bonus*"]
    for pattern in bonus_patterns:
        video_files.extend(video_dir.glob(f"{pattern}.mp4"))
        video_files.extend(video_dir.glob(f"{pattern}.mov"))
    
    video_files = list(set(video_files))  # Remove duplicates
    
    print(f"\n🎬 Found {len(video_files)} videos to process")
    print("="*60)
    
    results = {}
    
    for idx, video_path in enumerate(video_files, 1):
        print(f"\n[{idx}/{len(video_files)}] Processing: {video_path.name}")
        try:
            pgn = chess_move(str(video_path))
            results[video_path.name] = pgn
            print(f"  ✅ {len(pgn.split('.')) if pgn else 0} moves")
        except Exception as e:
            print(f"  ❌ Error: {str(e)}")
            results[video_path.name] = ""
    
    # Create CSV
    df = create_submission_csv(results, output_csv)
    
    print(f"\n📋 Sample results:")
    print(df.head())
    
    return df

# Example usage (uncomment to run):
video_directory = "/kaggle/working/"  # Update this path
df = process_all_videos_to_csv(video_directory, "/kaggle/working/wrong_name_submission.csv")

In [ ]:
import pandas as pd

# 1. Load the CSV
df = pd.read_csv("/kaggle/working/wrong_name_submission.csv")

# 2. CRITICAL STEP: Remove invisible spaces from the filename column
# This fixes the "it doesn't work" issue if there are hidden spaces
df['row_id'] = df['row_id'].str.strip()

# 3. Replace the name
# We look for the exact string now that spaces are cleaned
old_name = "(Bonus)Long_Video_label.mp4"
new_name = "(Bonus)Long_video_student.mp4"

# Use .loc to update specifically that row
df.loc[df['row_id'] == old_name, 'row_id'] = new_name

# 4. Verification Print
# This checks if the new name exists in the dataframe now
if len(df[df['row_id'] == new_name]) > 0:
    print("SUCCESS: The name has been changed.")
else:
    print("WARNING: Name not found. Please check if the file path is correct.")

# 5. Save to a new CSV file
df.to_csv("/kaggle/working/submissions.csv", index=False)